# Predicting the Price of the S&P500 Index with Linear Regression

In this project, we worked with data from the S&P500 Index. The S&P500 is a stock market index. We used historical data on the price of the S&P500 Index to make predictions about future prices.

We'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

The columns of the dataset are:

* Date -- The date of the record.
* Open -- The opening price of the day (when trading starts).
* High -- The highest trade price during the day.
* Low -- The lowest trade price during the day.
* Close -- The closing price for the day (when trading is finished).
* Volume -- The number of shares traded.
* Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions.

We'll be using this dataset to develop a predictive model. You'll train the model with data from 1950-2012 and try to make predictions from 2013-2015.

Note: You shouldn't make trades with any models developed in this lesson. Trading stocks has risks and nothing in this lesson constitutes stock trading advice.

In [113]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [114]:
#Read in the data set
df = pd.read_csv(r'C:\Users\henye\Downloads\sphist.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [115]:
#Sort by the date column in ascending order
df = df.sort_values("Date", ascending=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000,17.08


Add features that could be helpful for machine learning

We will use Pandas Time series tools: Rolling function to set the window equal to the past 5 trading days to compute the indicators. 
This adds in NaN values for any row where there aren't enough historical trading days to do the computation.

Note: We could use also indicators with 30 or 365 days of historical data. For the last case, since our dataset starts on 1950-01-03, for any rows that fall before 1951-01-03 we wouldn't have enough historical data to compute all the indicators. For this reason we would need to remove these rows before splitting the data as followed:

removing any rows from df that occur before 1951-01-03
df = df[df['Date'] >= datetime(year=1951, month=1, day=3)]
df.dropna(axis=0)
df.head()

In [116]:
df['5 Days Open'] = df['Open'].rolling(window=5).mean()
df['5 Days High'] = df['High'].rolling(window=5).mean()
df['5 Days Low'] = df['Low'].rolling(window=5).mean()
df['5 Days Volume'] = df['Volume'].rolling(window=5).mean()
df['Year'] = df['Date'].apply(lambda x: x.year)

df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,16.660000,NaN,NaN,NaN,NaN,1950
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,16.850000,NaN,NaN,NaN,NaN,1950
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,16.930000,NaN,NaN,NaN,NaN,1950
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,16.980000,NaN,NaN,NaN,NaN,1950
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,17.080000,16.900,16.900,16.900,2046000.0,1950
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,17.030001,16.974,16.974,16.974,2226000.0,1950
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,17.090000,17.022,17.022,17.022,2374000.0,1950
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,16.760000,16.988,16.988,16.988,2458000.0,1950
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,16.670000,16.926,16.926,16.926,2722000.0,1950
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000,16.719999,16.854,16.854,16.854,2510000.0,1950


In [117]:
#Adding Day of week column and set it to categorical
df['DOW'] = df['Date'].apply(lambda x: x.weekday())

#Returning the dummy columns for the new DOW
dow_df = pd.get_dummies(df['DOW'])
df = pd.concat([df, dow_df], axis=1)
df = df.drop(['DOW'], axis=1)

df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000,16.66,NaN,NaN,NaN,NaN,1950,0,1,0,0,0
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000,16.85,NaN,NaN,NaN,NaN,1950,0,0,1,0,0
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000,16.93,NaN,NaN,NaN,NaN,1950,0,0,0,1,0
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000,16.98,NaN,NaN,NaN,NaN,1950,0,0,0,0,1
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000,17.08,16.9,16.9,16.9,2046000.0,1950,1,0,0,0,0


Because 'rolling' method above include the current date to compute, the current date include future knowledge, which means that the prediction will look not good in real world , so we shift all the values forward one day.
This way the values will be the mean of the previous 5 days

In [118]:
df['5 Days Open'] = df['5 Days Open'].shift(1)
df['5 Days High'] = df['5 Days High'].shift(1)
df['5 Days Low'] = df['5 Days Low'].shift(1)
df['5 Days Volume'] = df['5 Days Volume'].shift(1)
df.head(6)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,16.660000,NaN,NaN,NaN,NaN,1950,0,1,0,0,0
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,16.850000,NaN,NaN,NaN,NaN,1950,0,0,1,0,0
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,16.930000,NaN,NaN,NaN,NaN,1950,0,0,0,1,0
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,16.980000,NaN,NaN,NaN,NaN,1950,0,0,0,0,1
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,17.080000,NaN,NaN,NaN,NaN,1950,1,0,0,0,0
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,17.030001,16.9,16.9,16.9,2046000.0,1950,0,1,0,0,0


# Linear Regression

In [119]:
#Split dataset into train set and test set.
train_df = df[df['Date'] < datetime(year=2013, month=1, day=1)]
test_df = df[df['Date'] >= datetime(year=2013, month=1, day=1)]

#Feature columns
train_colums = ['5 Days Open', '5 Days Volume', '5 Days High', '5 Days Low', 'Year', 0, 1, 2, 3, 4]

In [120]:
# Perform linear regression.
lr = LinearRegression()
lr.fit(train_df[train_colums], train_df['Close'])
prediction = lr.predict(test_df[train_colums])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Model Evaluation

In [121]:
#Comparing the predicted price againts the real prices in our data bases

test_df_copy = test_df.copy()
test_df_copy['Predicted close price'] = prediction
test_df_copy['Difference'] = (test_df_copy['Close'] - test_df_copy['Predicted close price'])
test_df_copy['Accuracy(%)'] = (1-(abs(test_df_copy['Close'] - test_df_copy['Predicted close price']) / test_df_copy['Close']
)) * 100
test_df_copy = DataFrame(test_df_copy[['Date','Close','Predicted close price','Difference', 'Accuracy(%)']])

print(test_df_copy)

          Date        Close  Predicted close price  Difference  Accuracy(%)
738 2013-01-02  1462.420044            1409.290878   53.129166    96.367038
737 2013-01-03  1459.369995            1429.899615   29.470380    97.980609
736 2013-01-04  1466.469971            1438.455036   28.014935    98.089635
735 2013-01-07  1461.890015            1456.820228    5.069787    99.653203
734 2013-01-08  1457.150024            1469.353529  -12.203505    99.162509
..         ...          ...                    ...         ...          ...
4   2015-12-01  2102.629883            2081.013283   21.616600    98.971926
3   2015-12-02  2079.510010            2090.797168  -11.287158    99.457220
2   2015-12-03  2049.620117            2085.905615  -36.285498    98.229648
1   2015-12-04  2091.689941            2069.170510   22.519431    98.923386
0   2015-12-07  2077.070068            2083.753457   -6.683389    99.678230

[739 rows x 5 columns]


In [104]:
# Error metrics.
mse = mean_squared_error(test_df['Close'], prediction)
rmse = np.sqrt(mse)

print('mse:',mse)
print('rmse:',rmse)
print('-'*60)
print('The S&P500 Index')

display(test_df_copy.sort_values(by=['Date'], ascending=False).head(20))

mse: 378.935638449297
rmse: 19.466269248351033
------------------------------------------------------------
The S&P500 Index


,Date,Close,Predicted close price,Difference,Accuracy(%)
0,2015-12-07,2077.070068,2083.753457,-6.683389,99.678230
1,2015-12-04,2091.689941,2069.170510,22.519431,98.923386
2,2015-12-03,2049.620117,2085.905615,-36.285498,98.229648
3,2015-12-02,2079.510010,2090.797168,-11.287158,99.457220
4,2015-12-01,2102.629883,2081.013283,21.616600,98.971926
5,2015-11-30,2080.409912,2089.912592,-9.502680,99.543230
6,2015-11-27,2090.110107,2087.914387,2.195720,99.894947
7,2015-11-25,2088.870117,2097.554893,-8.684776,99.584236
8,2015-11-24,2089.139893,2095.813837,-6.673944,99.680541
9,2015-11-23,2086.590088,2097.850319,-11.260231,99.460352
